This example mirrors [dragCallback.html](https://niivue.com/demos/features/dragCallback.html).

In [ ]:
from pathlib import Path
from ipyniivue import download_dataset

BASE_API_URL = "https://niivue.com/demos/images/"
DATA_FOLDER = Path("images")

# Download data for example
download_dataset(
    BASE_API_URL,
    DATA_FOLDER,
    files=[
        "mni152.nii.gz",
    ],
)

In [ ]:
import ipywidgets
import ipyniivue

# Initialize NiiVue
nv = ipyniivue.NiiVue()

# Set defaults
nv.opts.loading_text = "there are no images"
nv.opts.back_color = [1, 1, 1, 1]
nv.opts.show_3d_crosshair = True
nv.opts.limit_frames_4d = 3
nv.opts.is_radiological_convention = False
nv.opts.slice_type = ipyniivue.SliceType.MULTIPLANAR

# Location label
location_label = ipywidgets.Label(value="")


@nv.on_location_change
def handle_location_change(data):
    """Handle location change."""
    if "string" in data:
        location_label.value = data["string"]


@nv.on_drag_release
def handle_drag_release(info):
    """Handle drag release."""
    tile_idx = info.get("tile_idx", -1)
    if tile_idx < 0:
        location_label.value = "Invalid drag"
    else:
        ax_cor_sag = info.get("ax_cor_sag")
        mm_length = round(info.get("mm_length", 0))
        v_start = info.get("vox_start", [0, 0, 0])
        v_end = info.get("vox_end", [0, 0, 0])

        msg = (
            f"Tile: {tile_idx} Orient: {ax_cor_sag} Length:{mm_length} "
            f"x:{v_start[0]}..{v_end[0]} "
            f"y:{v_start[1]}..{v_end[1]} "
            f"z:{v_start[2]}..{v_end[2]}"
        )
        location_label.value = msg


# Drag modes dropdown
drag_modes = {
    "none": ipyniivue.DragMode.NONE,
    "contrast": ipyniivue.DragMode.CONTRAST,
    "measurement": ipyniivue.DragMode.MEASUREMENT,
    "pan/zoom": ipyniivue.DragMode.PAN,
    "slicer3D": ipyniivue.DragMode.SLICER_3D,
    "callbackOnly": ipyniivue.DragMode.CALLBACK_ONLY,
    "roiSelection": ipyniivue.DragMode.ROI_SELECTION,
    "angle": ipyniivue.DragMode.ANGLE,
}

drag_mode_dropdown = ipywidgets.Dropdown(
    options=drag_modes, value=ipyniivue.DragMode.CONTRAST, description="Drag mode:"
)


def on_drag_mode_change(change):
    """Handle drag mode change."""
    nv.opts.drag_mode = change["new"]


drag_mode_dropdown.observe(on_drag_mode_change, names="value")

# Load volume
nv.load_volumes([{"path": DATA_FOLDER / "mni152.nii.gz"}])

# Display all
ipywidgets.VBox([drag_mode_dropdown, nv, location_label])